# Data Warehouse Medicare Texas QA - Claim Diag

## Initialization

Just loading packages that will be used and initializing connection to GP DB.

In [1]:
import pandas as pd
import sys
import psycopg2
from tqdm import tqdm
sys.path.append('H:/uth_helpers')
from db_utils import get_dsn

In [2]:
connection = psycopg2.connect(get_dsn())
connection.autocommit = True

## Row Count and Claim Count

Similar to the member_enrollment_monthly table, the claim_header table has the row count should equal the number of unique claims.

For this table, we extract claim data from the s, o, and f tables from the mcrten schema. 


In [3]:
query = ''' drop table if exists qa_reporting.dw_mcrt_claim_diag_counts;
create table qa_reporting.dw_mcrt_claim_diag_counts
(
    calendar_year int,
    dw_row_count bigint,
    src_row_count bigint,
    row_count_diff bigint,
    row_count_percentage float,
    dw_uth_clm_id_count bigint,
    dw_src_clm_id_count bigint,
    src_clm_count bigint,
    clm_count_diff bigint,
    clm_count_percentage float,
    dw_uth_mbr_id_count bigint,
    dw_src_mbr_id_count bigint,
    src_mbr_count bigint,
    mbr_count_diff bigint,
    mbr_count_percentage float,
    date_generated date
);
'''

with connection.cursor() as cursor:
    cursor.execute(query)

In [4]:
with connection.cursor() as cursor:
    query = f'''
    insert into qa_reporting.dw_mcrt_claim_diag_counts
    (calendar_year, dw_row_count, dw_uth_clm_id_count, date_generated)
    select year, count(*), count(distinct uth_claim_id), current_date
    from dw_staging.mcrt_claim_diag
    where diag_position = '1'
    group by 1
    '''
    
    cursor.execute(query)

    query = f'''
    update qa_reporting.dw_mcrt_claim_diag_counts b
    set dw_src_clm_id_count = count
    from (
        select year,  count(distinct claim_id_src) as count 
        from dw_staging.mcrt_claim_diag
    group by 1) a
    where a.year = b.calendar_year
    '''

    cursor.execute(query)

    query = f'''
    update qa_reporting.dw_mcrt_claim_diag_counts b
    set dw_uth_mbr_id_count = count
    from (
        select year, count(distinct uth_member_id) as count 
        from dw_staging.mcrt_claim_diag
    group by 1) a
    where a.year = b.calendar_year
    '''

    cursor.execute(query)
    
    query = f'''
    update qa_reporting.dw_mcrt_claim_diag_counts b
    set dw_src_mbr_id_count = count
    from (
        select year, count(distinct member_id_src) as count 
        from dw_staging.mcrt_claim_diag
    group by 1) a
    where a.year = b.calendar_year
    '''

    cursor.execute(query)

In [21]:
with connection.cursor() as cursor:
    query = '''
    with clms as (
        select extract(year from clm_from_dt::date) as year, bene_id, clm_id
        from medicare_texas.hha_base_claims_k
        where icd_dgns_cd1 is not null
        union
        select extract(year from clm_from_dt::date) as year, bene_id, clm_id
        from medicare_texas.outpatient_base_claims_k
        where icd_dgns_cd1 is not null
        union
        select extract(year from clm_from_dt::date) as year, bene_id, clm_id
        from medicare_texas.dme_claims_k
        where icd_dgns_cd1 is not null
        union
        select extract(year from clm_from_dt::date)as year, bene_id, clm_id
        from medicare_texas.inpatient_base_claims_k
        where icd_dgns_cd1 is not null
        union
        select extract(year from clm_from_dt::date) as year, bene_id, clm_id
        from medicare_texas.bcarrier_claims_k
        where icd_dgns_cd1 is not null
        union
        select extract(year from clm_from_dt::date) as year, bene_id, clm_id
        from medicare_texas.hospice_base_claims_k
        where icd_dgns_cd1 is not null
        union
        select extract(year from clm_from_dt::date) as year, bene_id, clm_id
        from medicare_texas.snf_base_claims_k
        where icd_dgns_cd1 is not null
    ),
    clm_counts as (
        select year, count(*) row_count, count(distinct bene_id) pat_count, count(distinct clm_id) clm_count
        from clms
        group by 1
    )
    update qa_reporting.dw_mcrt_claim_diag_counts a
    set src_row_count = b.row_count,
    row_count_diff = a.dw_row_count - b.row_count,
    row_count_percentage = 100. * abs(a.dw_row_count - b.row_count) / b.row_count,
    src_clm_count = b.clm_count,
    clm_count_diff = a.dw_uth_clm_id_count - b.clm_count,
    clm_count_percentage = 100. * abs(a.dw_uth_clm_id_count - b.clm_count) / b.clm_count,
    src_mbr_count = b.pat_count,
    mbr_count_diff = a.dw_uth_mbr_id_count - b.pat_count,
    mbr_count_percentage = 100. * abs(a.dw_uth_mbr_id_count - b.pat_count) / b.pat_count
    from clm_counts b
    where a.calendar_year = b.year
    ;
    '''

    cursor.execute(query)

In [6]:
query = '''select * from qa_reporting.dw_mcrt_claim_diag_counts;'''

df = pd.read_sql(query, con=connection)
df.sort_values(['clm_count_percentage'], ascending=False)

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,calendar_year,dw_row_count,src_row_count,row_count_diff,row_count_percentage,dw_uth_clm_id_count,dw_src_clm_id_count,src_clm_count,clm_count_diff,clm_count_percentage,dw_uth_mbr_id_count,dw_src_mbr_id_count,src_mbr_count,mbr_count_diff,mbr_count_percentage,date_generated
19,2021,76648119,76784526.0,-136407.0,0.177649,76648119,76648119,76784526.0,-136407.0,0.177649,3398097,3398097,3398935.0,-838.0,0.024655,2023-12-18
21,2014,78590794,78623020.0,-32226.0,0.040988,78590794,78590794,78623020.0,-32226.0,0.040988,2520193,2520193,2521006.0,-813.0,0.032249,2023-12-18
15,2018,78096491,78125243.0,-28752.0,0.036802,78096491,78096491,78125243.0,-28752.0,0.036802,2508827,2508827,2509887.0,-1060.0,0.042233,2023-12-18
7,2017,80780357,80799923.0,-19566.0,0.024215,80780357,80780357,80799923.0,-19566.0,0.024215,2549118,2549118,2547302.0,1816.0,0.071291,2023-12-18
2,2019,77337755,77349845.0,-12090.0,0.015630,77337755,77337755,77349845.0,-12090.0,0.015630,2456512,2456512,2453152.0,3360.0,0.136967,2023-12-18
10,2016,80516454,80528127.0,-11673.0,0.014496,80516454,80516454,80528127.0,-11673.0,0.014496,2546813,2546813,2547595.0,-782.0,0.030696,2023-12-18
12,2015,79304963,79315887.0,-10924.0,0.013773,79304963,79304963,79315887.0,-10924.0,0.013773,2536400,2536400,2537197.0,-797.0,0.031413,2023-12-18
16,2020,72649727,72652706.0,-2979.0,0.004100,72649727,72649727,72652706.0,-2979.0,0.004100,2492470,2492470,2493287.0,-817.0,0.032768,2023-12-18
0,2003,1,NaN,NaN,NaN,1,1,NaN,NaN,NaN,1,1,NaN,NaN,NaN,2023-12-18
1,2009,7,NaN,NaN,NaN,7,7,NaN,NaN,NaN,7,7,NaN,NaN,NaN,2023-12-18


## Diagnosis Codes

Here we will check if we have valid ICD Diagnosis codes using our reference tables.

In [7]:
query = '''drop table if exists qa_reporting.dw_mcrt_diag_counts;
select year, diag_cd, count(*) as diag_count
into qa_reporting.dw_mcrt_diag_counts
from dw_staging.mcrt_claim_diag
group by 1,2;
'''

with connection.cursor() as cursor:
    cursor.execute(query)

In [8]:
diag_cd_df = pd.read_sql('select * from qa_reporting.dw_mcrt_diag_counts;', con=connection)
diag_cd_df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,diag_cd,diag_count
0,2019,S31159D,10
1,2019,P152,10
2,2020,S88021S,22
3,2015,S62649A,31
4,2021,Q0702,140
...,...,...,...
356465,2021,M9111,66
356466,2017,S06307A,11
356467,2015,E872,43489
356468,2021,M1A4610,9


We see that we have a lot of claims with diagnosis codes that are not in our reference list. If we take a further look at some of these codes, we see that in our reference list, there is at least one more digit missing from these codes. The missing digits helps specify the diagnosis code.

In [9]:
query = '''
select a.*
from qa_reporting.dw_mcrt_diag_counts a
left join reference_tables.ref_cms_icd_cm_codes b
on a.diag_cd = cd_value
where b.cd_value is null;
'''

invalid_diag_df = pd.read_sql(query, con=connection)
invalid_diag_df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,diag_cd,diag_count
0,2015,M45814,1
1,2015,R6289,1
2,2016,R629,1
3,2017,G150,1
4,2019,E359,1
...,...,...,...
2625,2014,5302,1
2626,2015,I419,3
2627,2018,J649,1
2628,2015,2962,2


Overall, the number of invalid diagnosis codes is neglible compared to the overall number of diagnosis codes in the claim_diag table.

In [10]:
invalid_diag_df.groupby('year')['diag_count'].sum()

year
2014    1759
2015    1874
2016     414
2017     285
2018     260
2019     518
2020     137
2021      39
Name: diag_count, dtype: int64

In [11]:
diag_comp_df = pd.DataFrame({'overall_diag_count': diag_cd_df.groupby('year')['diag_count'].sum(),
                            'invalid_diag_count': invalid_diag_df.groupby('year')['diag_count'].sum(),
                            'valid_diag_count': diag_cd_df.groupby('year')['diag_count'].sum() - invalid_diag_df.groupby('year')['diag_count'].sum()})
diag_comp_df.loc[diag_comp_df['valid_diag_count'].isna(),'valid_diag_count'] = diag_comp_df.loc[diag_comp_df['valid_diag_count'].isna(),'overall_diag_count']
diag_comp_df['valid_diag_count'] =  diag_comp_df['valid_diag_count'].astype(int)
diag_comp_df['invalid_to_valid_percent'] = 100. * diag_comp_df['invalid_diag_count'] / diag_comp_df['valid_diag_count']
diag_comp_df  

,overall_diag_count,invalid_diag_count,valid_diag_count,invalid_to_valid_percent
year,,,,
2001,99,NaN,99,NaN
2002,79,NaN,79,NaN
2003,9,NaN,9,NaN
2004,26,NaN,26,NaN
2005,53,NaN,53,NaN
2006,22,NaN,22,NaN
2007,57,NaN,57,NaN
2008,9,NaN,9,NaN
2009,62,NaN,62,NaN


In [12]:
invalid_diag_df.groupby('year').max()

,diag_cd,diag_count
year,,
2014,e9342,218
2015,m1712,185
2016,Z9982,28
2017,Z995,61
2018,ZB53,93
2019,ZOOOO,222
2020,Z99814,7
2021,XXX,7


## Diagnosis Code Position

In [13]:
query = '''drop table if exists qa_reporting.dw_mcrt_diag_position;
select year, diag_position, count(distinct uth_claim_id) as claim_count
into qa_reporting.dw_mcrt_diag_position
from dw_staging.mcrt_claim_diag
group by 1,2;
'''
with connection.cursor() as cursor:
    cursor.execute(query)


In [14]:
diag_position_df = pd.read_sql('select * from qa_reporting.dw_mcrt_diag_position;', con=connection)
diag_position_df.sort_values(['year', 'diag_position'])

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,diag_position,claim_count
262,2001,1,7
249,2001,10,6
139,2001,11,4
49,2001,12,4
445,2001,13,3
...,...,...,...
160,2022,5,299
407,2022,6,282
226,2022,7,248
245,2022,8,214


In [15]:
diag_position_df['diag_position'].unique()

array(['13', '2', '21', '4', '16', '1', '5', '6', '15', '8', '19', '17',
       '20', '14', '12', '22', '9', '10', '18', '23', '11', '7', '25',
       '3', '24'], dtype=object)

Checking if the counts for the diag_position are correct. The higher the diag_position is, the less counts there should be. If we sort the counts of the diag_position by year and assign their order, this value should match with the diag_position value.

In [16]:
diag_position_df['row_rank'] = diag_position_df.sort_values(['year', 'claim_count'], ascending=[True, False]).groupby(['year']).cumcount()+1
diag_position_df['position_check'] = diag_position_df['row_rank'] == diag_position_df['diag_position'].astype(int)
diag_position_df[~diag_position_df['position_check']]

,year,diag_position,claim_count,row_rank,position_check
0,2005,13,2,12,False
4,2002,16,2,15,False
7,2003,5,1,1,False
8,2011,6,27,5,False
9,2007,15,2,9,False
...,...,...,...,...,...
457,2002,8,4,9,False
467,2006,2,2,8,False
468,2004,21,1,23,False
474,2005,5,4,10,False


In [17]:
diag_position_df[~diag_position_df['position_check']].sort_values('claim_count', ascending=False)

,year,diag_position,claim_count,row_rank,position_check
8,2011,6,27,5,False
163,2011,5,27,6,False
439,2010,6,15,7,False
78,2010,7,15,6,False
240,2010,14,8,15,False
...,...,...,...,...,...
116,2003,8,1,4,False
113,2004,15,1,10,False
112,2004,6,1,9,False
107,2003,4,1,3,False


## ICD Version

In [19]:
query = '''
drop table if exists qa_reporting.dw_mcrt_icd_version_count;

select year, icd_version, count(distinct uth_claim_id) as claim_count
into qa_reporting.dw_mcrt_icd_version_count
from dw_staging.mcrt_claim_diag
group by 1,2;
'''
with connection.cursor() as cursor:
    cursor.execute(query)

In [20]:
icd_version_df = pd.read_sql('select * from qa_reporting.dw_mcrt_icd_version_count;', con=connection)
icd_version_df.sort_values(['year', 'icd_version'])

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,icd_version,claim_count
15,2001,9,1
29,2001,None,6
18,2002,9,2
16,2002,None,6
40,2003,None,1
8,2004,9,2
14,2004,None,1
33,2005,9,2
36,2005,None,4
24,2006,None,2
